In [0]:
dbutils.widgets.text("catalog", "", "Catalog")
dbutils.widgets.text("schema", "", "Schema")
dbutils.widgets.text("volume", "", "Volume")
dbutils.widgets.text("source_folder", "", "Source Folder")

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Testando parametros via widgets
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
volume = dbutils.widgets.get("volume")
source_folder = dbutils.widgets.get("source_folder")

path_raw = f"dbfs:/Volumes/{catalog}/{schema}/{volume}/{source_folder}"

print(f"Path de leitura: {path_raw}")



Path de leitura: dbfs:/Volumes/ted_dev/dev_marcio_fernandes/raw/extracted_from_db


In [0]:
endpoints = [
    f.name.rstrip("/")
    for f in dbutils.fs.ls(path_raw)
    if f.isDir()
]

print (f"Endpoints: {endpoints}")

Endpoints: ['HumanResources-Department', 'HumanResources-Employee', 'HumanResources-EmployeeDepartmentHistory', 'HumanResources-EmployeePayHistory', 'HumanResources-JobCandidate', 'HumanResources-Shift', 'Person-Address', 'Person-AddressType', 'Person-BusinessEntity', 'Person-BusinessEntityAddress', 'Person-BusinessEntityContact', 'Person-ContactType', 'Person-CountryRegion', 'Person-EmailAddress', 'Person-Person', 'Person-PersonPhone', 'Person-PhoneNumberType', 'Person-StateProvince', 'Production-BillOfMaterials', 'Production-Location', 'Production-Product', 'Production-ProductCategory', 'Production-ProductCostHistory', 'Production-ProductDescription', 'Production-ProductListPriceHistory', 'Production-ProductSubcategory', 'Production-TransactionHistory', 'Purchasing-ProductVendor', 'Purchasing-PurchaseOrderDetail', 'Purchasing-PurchaseOrderHeader', 'Purchasing-ShipMethod', 'Purchasing-Vendor', 'Sales-CountryRegionCurrency', 'Sales-CreditCard', 'Sales-Currency', 'Sales-CurrencyRate', '

In [0]:
for endpoint in endpoints:
    parquet_path = f"/Volumes/{catalog}/{schema}/{volume}/{source_folder}/{endpoint}/"
    table_name = f"{catalog}.{schema}.raw_db_{endpoint.replace('-', '_')}"
    
    sql = f"""
        CREATE OR REPLACE TABLE {table_name}
        AS
        SELECT * FROM parquet.`{parquet_path}`
    """

    print(f"Criando tabela: {table_name}")
    spark.sql(sql)